In [ ]:
import glob
import math
import uproot
import numpy as np
import pandas as pd
import pkgutil
import uproot_methods

import functions_giammi as fugi

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
%matplotlib inline

import re

#for the cartesian product
import itertools

#to smooth the MFPADs
import scipy as sp
import scipy.ndimage

#to create the mesh
import triangulation as tr
from scipy.spatial import Delaunay
from pyntcloud import PyntCloud, structures

import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS

#to see which classes have been defined so far
[modname for importer, modname, ispkg in pkgutil.walk_packages(uproot_methods.classes.__path__)]

#Select between CH9 and CH11
# path = "angular_distr_el/CH9/"
path = "angular_distr_el/CH11/"

#move first the ID you need
IDs = ["ID1_mol_e0_valid/EN_gate/","ID_ALL_mol_e0_valid/EN_gate/","ID2_mol_e0_valid/EN_gate/","ID3_mol_e0_valid/EN_gate/"]
dirs = ["MFPADs_multinew","MFPADs_multinew_phi"]

loc=path+IDs[0]+dirs[0]

##550eV - 11.5eV CH9
#update nov Kilian binning (36,18)
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_CH9_550eV_CR_Knov.root")
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_CH9_550eV_CL_Knov.root")
# fileSCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_CH9_550eV_CR_Knov.root")
# fileSCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_CH9_550eV_CL_Knov.root")

##550eV - 11.5eV CH11
#update nov Kilian binning (36,18)
fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_CH11_550eV_CR_Knov.root")
fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_CH11_550eV_CL_Knov.root")
fileSCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_CH11_550eV_CR_Knov.root")
fileSCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_CH11_550eV_CL_Knov.root")

#low new binning (24,12)
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_550eV_CR_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_550eV_CL_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")
# fileSCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_550eV_CR_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")
# fileSCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_550eV_CL_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")

# #high neew binning (36,18)
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_550eV_CR_newEfield_ALLCH_MFPAD_30_t0_higherbins.root")
# fileSCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_550eV_CR_newEfield_ALLCH_MFPAD_30_t0_higherbins.root")
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_550eV_CL_newEfield_ALLCH_MFPAD_30_t0_higherbins.root")
# fileSCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_550eV_CL_newEfield_ALLCH_MFPAD_30_t0_higherbins.root")

#546eV - 8.2eV
#low new binning
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_546.5eV_CR_9600-3700ns_multiCH9_MFPAD_30_t0.root")
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_546.5eV_CL_9600-3700ns_multiCH9_MFPAD_30_t0.root")

#544eV - 6.1eV
#low new binning
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_544.5eV_CR_9600-3700ns_multiCH9_MFPAD_30_t0.root")
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_544.5eV_CL_9600-3700ns_multiCH9_MFPAD_30_t0.root")

#542eV - 4eV
#low new binning
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_542eV_CR_9600-3700ns_multiCH9_MFPAD_30_t0.root")
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_542eV_CL_9600-3700ns_multiCH9_MFPAD_30_t0.root")

#541eV 3eV
#low new binning
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_541eV_CR_9600-3700ns_multiCH9_MFPAD_30_t0.root")
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_541eV_CL_9600-3700ns_multiCH9_MFPAD_30_t0.root")

#new uproot4
# https://uproot.readthedocs.io/en/latest/basic.html
d=dict(fileRCR[loc].classnames()) # shows the type of each item
# k=fileRCR[loc].keys() #same as .classnames()
check = fileRCR[loc+"/MFPAD3D_engate_costheta_-1.00_phi_-180"]
d1=type(check)
check.all_members; #all members of the single histogram

In [ ]:
import matplotlib as mpl
from matplotlib import cm
import matplotlib.colors

#the following converts a cmap into plotly language 
magma_cmap = mpl.cm.get_cmap('magma')
seismic_cmap = mpl.cm.get_cmap('seismic')

magma_rgb = []
seismic_rgb = []
norm = mpl.colors.Normalize(vmin=0, vmax=255)

for i in range(0, 255):
       k = mpl.colors.colorConverter.to_rgb(magma_cmap(norm(i)))
       magma_rgb.append(k)

for i in range(0, 255):
       k = mpl.colors.colorConverter.to_rgb(seismic_cmap(norm(i)))
       seismic_rgb.append(k)

def matplotlib_to_plotly(cmap, pl_entries):
    h = 1.0/(pl_entries-1)
    pl_colorscale = []

    for k in range(pl_entries):
        C = list(map(np.uint8, np.array(cmap(k*h)[:3])*255))
        pl_colorscale.append([k*h, 'rgb'+str((C[0], C[1], C[2]))])

    return pl_colorscale

Magma_r = matplotlib_to_plotly(magma_cmap, 255)
Seismic_r = matplotlib_to_plotly(seismic_cmap, 255)

#temperature cmap
colors = [[0, "blue"],
          [0.5, "white"],
          [0.75, "yellow"],
          [1, "red"]]

cmap_temp = matplotlib.colors.LinearSegmentedColormap.from_list("temperature_philipp", colors)
cmap_temp_go = go.color_continuous_scale=[(0, "blue"), (0.5, "white"), (0.75, "yellow"), (1,"red")]

In [ ]:
# initilialises a list
MFPAD_xy=[];
ctheta_cred=[];

MFPAD_RCR=[]; ctheta_RCR=[]; ctheta_xaxis_RCR=[]; cosphi_RCR=[];
MFPAD_RCL=[]; ctheta_RCL=[]; ctheta_xaxis_RCL=[]; cosphi_RCL=[];

fugi.import_MFPAD(fileRCR, loc, MFPAD_RCR, cosphi_RCR, MFPAD_xy, ctheta_RCR, ctheta_xaxis_RCR, ctheta_cred)
fugi.import_MFPAD(fileRCL, loc, MFPAD_RCL, cosphi_RCL, MFPAD_xy, ctheta_RCL, ctheta_xaxis_RCL, ctheta_cred)

# converting into nparray
MFPAD_RCR=np.array(MFPAD_RCR); ctheta_RCR=np.array(ctheta_RCR); ctheta_xaxis_RCR=np.array(ctheta_xaxis_RCR[0]); cosphi_RCR=np.array(cosphi_RCR);
MFPAD_RCL=np.array(MFPAD_RCL); ctheta_RCL=np.array(ctheta_RCL); ctheta_xaxis_RCL=np.array(ctheta_xaxis_RCL[0]); cosphi_RCL=np.array(cosphi_RCL);

if fileSCL and fileSCR is not None:
    MFPAD_SCR=[]; ctheta_SCR=[]; ctheta_xaxis_SCR=[]; cosphi_SCR=[];
    MFPAD_SCL=[]; ctheta_SCL=[]; ctheta_xaxis_SCL=[]; cosphi_SCL=[];
    
    fugi.import_MFPAD(fileSCR, loc, MFPAD_SCR, cosphi_SCR, MFPAD_xy, ctheta_SCR, ctheta_xaxis_SCR, ctheta_cred)
    fugi.import_MFPAD(fileSCL, loc, MFPAD_SCL, cosphi_SCL, MFPAD_xy, ctheta_SCL, ctheta_xaxis_SCL, ctheta_cred)
    
    # converting into nparray
    MFPAD_SCR=np.array(MFPAD_SCR); ctheta_SCR=np.array(ctheta_SCR); ctheta_xaxis_SCR=np.array(ctheta_xaxis_SCR[0]); cosphi_SCR=np.array(cosphi_SCR);
    MFPAD_SCL=np.array(MFPAD_SCL); ctheta_SCL=np.array(ctheta_SCL); ctheta_xaxis_SCL=np.array(ctheta_xaxis_SCL[0]); cosphi_SCL=np.array(cosphi_SCL);

ctheta_cred=ctheta_cred[0];
cosphi=cosphi_RCR;

print("cosphi diagnostic: type=",type(cosphi), " shape= ", cosphi.shape)

go_phi=fugi.shift_func(cosphi[:,1]).tolist();
go_cos=fugi.shift_func(cosphi[:,0]).tolist();
print("go_phi length = ", len(go_phi))
print("go_cos length = ", len(go_cos))

#adjusting cosphi witha shifting to have equal to theory
cosphi_adj=[]
for phi,cos in zip(go_phi,go_cos):
    cosphi_adj.append((cos,phi))
cosphi_adj=np.around(np.array(cosphi_adj),3)

print("cosphi_adj: type=",type(cosphi_adj), " shape= ", cosphi_adj.shape)

phiM=(MFPAD_xy[0][0][1:] + MFPAD_xy[0][0][:-1])/2
cosM=(MFPAD_xy[0][1][1:] + MFPAD_xy[0][1][:-1])/2

print("MFPAD_xy shape = ", MFPAD_xy[0][0].shape)
print("phiM shape = ", phiM.shape)
print("cosM shape = ", cosM.shape)

ncosbin=MFPAD_RCR.shape[1]
nphibin=MFPAD_RCR.shape[2]

In [ ]:
go_phi
phiM

In [ ]:
print(MFPAD_RCR.shape)
print(MFPAD_RCR.max())
print(MFPAD_RCR.min())

In [ ]:
plt.contourf(phiM,cosM,MFPAD_RCR[0].T)
plt.legend(loc="best")
MFPAD_RCL[0].T.shape

In [ ]:
MFPAD_RCR_cos=fugi.sorting_array(MFPAD_RCR, cosphi_adj, phiM, cosM, 1)
MFPAD_RCL_cos=fugi.sorting_array(MFPAD_RCL, cosphi_adj, phiM, cosM, 1)
MFPAD_RCR_phi=fugi.sorting_array(MFPAD_RCR, cosphi_adj, phiM, cosM, 2)
MFPAD_RCL_phi=fugi.sorting_array(MFPAD_RCL, cosphi_adj, phiM, cosM, 2)

if fileSCL and fileSCR is not None:
    MFPAD_SCR_cos=fugi.sorting_array(MFPAD_SCR, cosphi_adj, phiM, cosM, 1)
    MFPAD_SCL_cos=fugi.sorting_array(MFPAD_SCL, cosphi_adj, phiM, cosM, 1)
    MFPAD_SCR_phi=fugi.sorting_array(MFPAD_SCR, cosphi_adj, phiM, cosM, 2)
    MFPAD_SCL_phi=fugi.sorting_array(MFPAD_SCL, cosphi_adj, phiM, cosM, 2)

In [ ]:
#flipping; axis=0 flips along cos(theta), axis=1 along phi
#there is a smart (and correct) flipping reccomended by Kilian
MFPAD_RCL_flip=[]
MFPAD_RCR_flip=[]

for i in range(72):
    temp1=np.flip(MFPAD_RCL[i], axis=0)
    temp2=np.flip(temp1, axis=1)
    MFPAD_RCL_flip.append(temp2)

MFPAD_RCL_flip=np.array(MFPAD_RCL_flip)

for i in range(72):
    temp1=np.flip(MFPAD_RCR[i], axis=0)
    temp2=np.flip(temp1, axis=1)
    MFPAD_RCR_flip.append(temp2)

MFPAD_RCR_flip=np.array(MFPAD_RCR_flip)

MFPAD_RCL_flip_cos=fugi.sorting_array(MFPAD_RCL_flip, cosphi_adj, phiM, cosM, 1)
MFPAD_RCL_flip_phi=fugi.sorting_array(MFPAD_RCL_flip, cosphi_adj, phiM, cosM, 2)

MFPAD_RCR_flip_cos=fugi.sorting_array(MFPAD_RCR_flip, cosphi_adj, phiM, cosM, 1)
MFPAD_RCR_flip_phi=fugi.sorting_array(MFPAD_RCR_flip, cosphi_adj, phiM, cosM, 2)


if fileSCL and fileSCR is not None:
    MFPAD_SCL_flip=[]
    MFPAD_SCR_flip=[]

    for i in range(72):
        temp1=np.flip(MFPAD_SCL[i], axis=0)
        temp2=np.flip(temp1, axis=1)
        MFPAD_SCL_flip.append(temp2)

    MFPAD_SCL_flip=np.array(MFPAD_SCL_flip)


    for i in range(72):
        temp1=np.flip(MFPAD_SCR[i], axis=0)
        temp2=np.flip(temp1, axis=1)
        MFPAD_SCR_flip.append(temp2)

    MFPAD_SCR_flip=np.array(MFPAD_SCR_flip)

    MFPAD_SCL_flip_cos=fugi.sorting_array(MFPAD_SCL_flip, cosphi_adj, phiM, cosM, 1)
    MFPAD_SCL_flip_phi=fugi.sorting_array(MFPAD_SCL_flip, cosphi_adj, phiM, cosM, 2)

    MFPAD_SCR_flip_cos=fugi.sorting_array(MFPAD_SCR_flip, cosphi_adj, phiM, cosM, 1)
    MFPAD_SCR_flip_phi=fugi.sorting_array(MFPAD_SCR_flip, cosphi_adj, phiM, cosM, 2)

In [ ]:
#smart flipping: coS and phi before?
MFPAD_sum_RCLCR=[]

#helicity: -cos(theta), phi+180
for i in range(6):
    for j in range(12):
        if j+6<=11:
            MFPAD_sum_RCLCR.append(MFPAD_RCL.reshape(12,6,ncosbin,nphibin)[j,i,:,:]+MFPAD_RCR.reshape(12,6,ncosbin,nphibin)[j+6,-i,:,:])
        else:
            MFPAD_sum_RCLCR.append(MFPAD_RCL.reshape(12,6,ncosbin,nphibin)[j,i,:,:]+MFPAD_RCR.reshape(12,6,ncosbin,nphibin)[j-6,-i,:,:])

MFPAD_sum_RCLCR=np.array(MFPAD_sum_RCLCR).reshape(72,ncosbin,nphibin)

MFPAD_sum_RCLCR_cos=fugi.sorting_array(MFPAD_sum_RCLCR, cosphi_adj, phiM, cosM, 1)
MFPAD_sum_RCLCR_phi=fugi.sorting_array(MFPAD_sum_RCLCR, cosphi_adj, phiM, cosM, 2)


if fileSCL and fileSCR is not None:
    MFPAD_sum_SCLCR=[]
    MFPAD_sum_SR=[]

    #helicity: -cos(theta), phi+180
    for i in range(6):
        for j in range(12):
            if j+6<=11:
                MFPAD_sum_SCLCR.append(MFPAD_SCL.reshape(12,6,ncosbin,nphibin)[j,i,:,:]+MFPAD_SCR.reshape(12,6,ncosbin,nphibin)[j+6,-i,:,:])
            else:
                MFPAD_sum_SCLCR.append(MFPAD_SCL.reshape(12,6,ncosbin,nphibin)[j,i,:,:]+MFPAD_SCR.reshape(12,6,ncosbin,nphibin)[j-6,-i,:,:])

    MFPAD_sum_SCLCR=np.array(MFPAD_sum_SCLCR).reshape(72,ncosbin,nphibin)

    MFPAD_sum_SCLCR_cos=fugi.sorting_array(MFPAD_sum_SCLCR, cosphi_adj, phiM, cosM, 1)
    MFPAD_sum_SCLCR_phi=fugi.sorting_array(MFPAD_sum_SCLCR, cosphi_adj, phiM, cosM, 2)
    
    #enantiomer: -cos(theta), -cos(theta)_photon
    for i in range(6):
        for j in range(12):
            #not sure about RCL. should be RCR instead
            MFPAD_sum_SR.append(MFPAD_sum_SCLCR.reshape(12,6,ncosbin,nphibin)[j,i,:,:]+np.flip(MFPAD_sum_RCLCR.reshape(12,6,ncosbin,nphibin)[j,-i,:,:], axis=0))

    MFPAD_sum_SR=np.array(MFPAD_sum_SR).reshape(72,ncosbin,nphibin)

    MFPAD_sum_SR_cos=fugi.sorting_array(MFPAD_sum_SR, cosphi_adj, phiM, cosM, 1)
    MFPAD_sum_SR_phi=fugi.sorting_array(MFPAD_sum_SR, cosphi_adj, phiM, cosM, 2)

In [ ]:
#Sum of all MFPADs
MFPAD_sumall_RCR_norm=[]
MFPAD_sumall_RCL_norm=[]

#sum within an experimemt
#single enantiomer and helicity
MFPAD_sumall_RCR=np.sum(MFPAD_RCR, axis=0)
MFPAD_sumall_RCL=np.sum(MFPAD_RCL, axis=0)

#normalised
for el in MFPAD_RCR:
    MFPAD_sumall_RCR_norm.append(el/np.linalg.norm(el,axis=1)[:,np.newaxis])
for el in MFPAD_RCL:
    MFPAD_sumall_RCL_norm.append(el/np.linalg.norm(el,axis=1)[:,np.newaxis])

MFPAD_sumall_RCR_norm=np.sum(np.array(MFPAD_sumall_RCR_norm),axis=0)
MFPAD_sumall_RCL_norm=np.sum(np.array(MFPAD_sumall_RCL_norm),axis=0)

#sum of two helicities
MFPAD_sumall_RCLCR=np.sum(MFPAD_sum_RCLCR, axis=0)
#normalised: is it ok to do this now instead of when summing up CL CR?
MFPAD_sumall_RCLCR_norm=np.sum(MFPAD_sum_RCLCR/np.linalg.norm(MFPAD_sum_RCLCR,axis=1)[:,np.newaxis], axis=0)

if fileSCL and fileSCR is not None:
    MFPAD_sumall_SCR_norm=[]
    MFPAD_sumall_SCL_norm=[]

    #sum within an experimemt
    #single enantiomer and helicity
    MFPAD_sumall_SCR=np.sum(MFPAD_SCR, axis=0)
    MFPAD_sumall_SCL=np.sum(MFPAD_SCL, axis=0)

    #normalised
    for el in MFPAD_SCR:
        MFPAD_sumall_SCR_norm.append(el/np.linalg.norm(el,axis=1)[:,np.newaxis])
    for el in MFPAD_SCL:
        MFPAD_sumall_SCL_norm.append(el/np.linalg.norm(el,axis=1)[:,np.newaxis])

    MFPAD_sumall_SCR_norm=np.sum(np.array(MFPAD_sumall_SCR_norm),axis=0)
    MFPAD_sumall_SCL_norm=np.sum(np.array(MFPAD_sumall_SCL_norm),axis=0)

    #sum of two helicities
    MFPAD_sumall_SCLCR=np.sum(MFPAD_sum_SCLCR, axis=0)
    #normalised: is it ok to do this now instead of when summing up CL CR?
    MFPAD_sumall_SCLCR_norm=np.sum(MFPAD_sum_SCLCR/MFPAD_sum_SCLCR.sum(axis=1)[:,np.newaxis], axis=0)
    
    #sum of two enantiomers
    MFPAD_sumall_SR=np.sum(MFPAD_sum_SR, axis=0)
    #normalised
    MFPAD_sumall_SR_norm=np.sum(MFPAD_sum_SR/MFPAD_sum_SR.sum(axis=1)[:,np.newaxis], axis=0)

In [ ]:
#SUM of flipped S enantiomers
fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
# fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
fig.subplots_adjust(hspace = .5, wspace=.5)

# WHATCH OUT! experimental data are saved in the form (cos(theta), phi) looping internally on cos(theta) -> subplots has to be filled column by column instead of row by row
# for i,ax in zip(range(72),axes.T.flatten()) :
for i,ax in zip(range(72),axes.T.flatten()) :
    #not the right way to add stuff: see changes above discussed with kilian
    # ax.contourf(phiM,cosM,fugi.smoothgauss(MFPAD_SCL_phi[i]+MFPAD_SCR_flip_phi[i]+MFPAD_RCR_phi[i]+MFPAD_RCL_flip_phi[i],0.3,0.6).T)
    ax.contourf(phiM,cosM,fugi.smoothgauss(MFPAD_SCL_phi[i]+MFPAD_SCR_flip_phi[i]+MFPAD_RCR_phi[i]+MFPAD_RCL_flip_phi[i],0.3,0.6).T)
    # ax.text(0, 0, cosphi[i].T, ha="center", va="center", color="red",fontsize=18)

newax = fig.add_subplot()
newax.patch.set_visible(False)
newax.minorticks_off()
newax.tick_params(which="both", direction='out', right=False, labelright=False)

newax.spines['bottom'].set_position(('outward', 45))
newax.spines['left'].set_position(('outward', 50))
newax.spines['right'].set_visible(False)
newax.spines['top'].set_visible(False)

newax.xaxis.set_ticks_position('bottom')
newax.xaxis.set_label_position('bottom')

newax.set_xticks(np.arange(-180,180,30))
newax.set_xlim([-180,180])
newax.set_ylim([-1,1])
newax.set_xlabel('phi_photon')
newax.set_ylabel('cos(theta)_photon')

fig.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
# fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
fig.subplots_adjust(hspace = .5, wspace=.5)

# WHATCH OUT! experimental data are saved in the form (cos(theta), phi) looping internally on cos(theta) -> subplots has to be filled column by column instead of row by row
# for i,ax in zip(range(72),axes.T.flatten()) :
for i,ax in zip(range(72),axes.T.flatten()) :
    ax.contourf(phiM,cosM,MFPAD_RCR_cos[i].T)
    # ax.text(0, 0, cosphi[i].T, ha="center", va="center", color="red",fontsize=18)

newax = fig.add_subplot()
newax.patch.set_visible(False)
newax.minorticks_off()
newax.tick_params(which="both", direction='out', right=False, labelright=False)

newax.spines['bottom'].set_position(('outward', 45))
newax.spines['left'].set_position(('outward', 50))
newax.spines['right'].set_visible(False)
newax.spines['top'].set_visible(False)

newax.xaxis.set_ticks_position('bottom')
newax.xaxis.set_label_position('bottom')

newax.set_xticks(np.arange(-180,180,30))
newax.set_xlim([-180,180])
newax.set_ylim([-1,1])
newax.set_xlabel('phi_photon')
newax.set_ylabel('cos(theta)_photon')

fig.tight_layout()
plt.show()

#just a test: as to be identical to the unsorted one

fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
#fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
fig.subplots_adjust(hspace = .5, wspace=.5)

#WHATCH OUT! experimental data are saved in the form (cos(theta), phi) looping internally on cos(theta) -> subplots has to be filled column by column instead of row by row
#for i,ax in zip(range(72),axes.T.flatten()) :
for i,ax in zip(range(72),axes.T.flatten()) :
    ax.contourf(phiM,cosM,MFPAD_RCR_phi[i].T)
    ax.text(0, 0, cosphi[i].T, ha="center", va="center", color="red",fontsize=18)
    
fig.tight_layout()
plt.show()

#unsorted version

fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
#fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
fig.subplots_adjust(hspace = .5, wspace=.5)

#WHATCH OUT! experimental data are saved in the form (cos(theta), phi) looping internally on cos(theta) -> subplots has to be filled column by column instead of row by row
#for i,ax in zip(range(72),axes.T.flatten()) :
for i,ax in zip(range(72),axes.T.flatten()) :
    ax.contourf(phiM,cosM,MFPAD_RCR[i].T)
    ax.text(0, 0, cosphi[i].T, ha="center", va="center", color="red",fontsize=18)
    
fig.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
#fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
fig.subplots_adjust(hspace = .5, wspace=.5)

#WHATCH OUT! experimental data are saved in the form (cos(theta), phi) looping internally on cos(theta) -> subplots has to be filled column by column instead of row by row
#for i,ax in zip(range(72),axes.T.flatten()) :
for i,ax in zip(range(72),axes.T.flatten()) :
    ax.imshow(fugi.smoothgauss(MFPAD_RCL_cos[i],0.3,0.6).T, extent=[phiM[0],phiM[-1],cosM[0],cosM[-1]], interpolation='quadric', cmap="viridis", aspect="auto")
    # ax.text(0, 0, cosphi[i].T, ha="center", va="center", color="red",fontsize=18)

newax = fig.add_subplot()
newax.patch.set_visible(False)
newax.minorticks_off()
newax.tick_params(which="both", direction='out', right=False, labelright=False)

newax.spines['bottom'].set_position(('outward', 45))
newax.spines['left'].set_position(('outward', 50))
newax.spines['right'].set_visible(False)
newax.spines['top'].set_visible(False)

newax.xaxis.set_ticks_position('bottom')
newax.xaxis.set_label_position('bottom')

newax.set_xticks(np.arange(-180,180,30))
newax.set_xlim([-180,180])
newax.set_ylim([-1,1])
newax.set_xlabel('phi_photon')
newax.set_ylabel('cos(theta)_photon')

fig.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
#fig, axes = plt.subplots(6,12, figsize=(25, 12), sharex='col', sharey='row')
fig.subplots_adjust(hspace = .5, wspace=.5)

#WHATCH OUT! experimental data are saved in the form (cos(theta), phi) looping internally on cos(theta) -> subplots has to be filled column by column instead of row by row
for i,ax in zip(range(72),axes.T.flatten()) :
    ax.contourf(phiM,cosM,fugi.smoothgauss(MFPAD_RCR_phi[i],0.3,0.6).T)
    # ax.text(0, 0, cosphi[i].T, ha="center", va="center", color="red",fontsize=18)
    
newax = fig.add_subplot()
newax.patch.set_visible(False)
newax.minorticks_off()
newax.tick_params(which="both", direction='out', right=False, labelright=False)

newax.spines['bottom'].set_position(('outward', 45))
newax.spines['left'].set_position(('outward', 50))
newax.spines['right'].set_visible(False)
newax.spines['top'].set_visible(False)

newax.xaxis.set_ticks_position('bottom')
newax.xaxis.set_label_position('bottom')

newax.set_xticks(np.arange(-180,180,30))
newax.set_xlim([-180,180])
newax.set_ylim([-1,1])
newax.set_xlabel('phi_photon')
newax.set_ylabel('cos(theta)_photon')

fig.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
#fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
fig.subplots_adjust(hspace = .5, wspace=.5)

#WHATCH OUT! experimental data are saved in the form (cos(theta), phi) looping internally on cos(theta) -> subplots has to be filled column by column instead of row by row
# for i,ax in zip(range(72),axes.T.flatten()) :
for i,ax in zip(range(72),axes.T.flatten()) :
    ax.contourf(phiM,cosM,fugi.smoothgauss(MFPAD_RCL_phi[i],0.3,0.6).T)
    #ax.text(0, 0, cosphi[i].T, ha="center", va="center", color="red",fontsize=18)
    
newax = fig.add_subplot()
newax.patch.set_visible(False)
newax.minorticks_off()
newax.tick_params(which="both", direction='out', right=False, labelright=False)

newax.spines['bottom'].set_position(('outward', 45))
newax.spines['left'].set_position(('outward', 50))
newax.spines['right'].set_visible(False)
newax.spines['top'].set_visible(False)

newax.xaxis.set_ticks_position('bottom')
newax.xaxis.set_label_position('bottom')

newax.set_xticks(np.arange(-180,180,30))
newax.set_xlim([-180,180])
newax.set_ylim([-1,1])
newax.set_xlabel('phi_photon')
newax.set_ylabel('cos(theta)_photon')

fig.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
#fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
fig.subplots_adjust(hspace = .5, wspace=.5)

#WHATCH OUT! experimental data are saved in the form (cos(theta), phi) looping internally on cos(theta) -> subplots has to be filled column by column instead of row by row
#for i,ax in zip(range(72),axes.T.flatten()) :
for i,ax in zip(range(72),axes.T.flatten()) :
    ax.contourf(phiM,cosM,fugi.smoothgauss(MFPAD_SCR_phi[i],0.3,0.6).T)
    #ax.text(0, 0, cosphi[i].T, ha="center", va="center", color="red",fontsize=18)
    
newax = fig.add_subplot()
newax.patch.set_visible(False)
newax.minorticks_off()
newax.tick_params(which="both", direction='out', right=False, labelright=False)

newax.spines['bottom'].set_position(('outward', 45))
newax.spines['left'].set_position(('outward', 50))
newax.spines['right'].set_visible(False)
newax.spines['top'].set_visible(False)

newax.xaxis.set_ticks_position('bottom')
newax.xaxis.set_label_position('bottom')

newax.set_xticks(np.arange(-180,180,30))
newax.set_xlim([-180,180])
newax.set_ylim([-1,1])
newax.set_xlabel('phi_photon')
newax.set_ylabel('cos(theta)_photon')

fig.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
#fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
fig.subplots_adjust(hspace = .5, wspace=.5)

#WHATCH OUT! experimental data are saved in the form (cos(theta), phi) looping internally on cos(theta) -> subplots has to be filled column by column instead of row by row
#for i,ax in zip(range(72),axes.T.flatten()) :
for i,ax in zip(range(72),axes.T.flatten()) :
    ax.contourf(phiM,cosM,fugi.smoothgauss(MFPAD_SCL_phi[i],0.3,0.6).T)    
    #ax.text(0, 0, cosphi[i].T, ha="center", va="center", color="red",fontsize=18)
    
newax = fig.add_subplot()
newax.patch.set_visible(False)
newax.minorticks_off()
newax.tick_params(which="both", direction='out', right=False, labelright=False)

newax.spines['bottom'].set_position(('outward', 45))
newax.spines['left'].set_position(('outward', 50))
newax.spines['right'].set_visible(False)
newax.spines['top'].set_visible(False)

newax.xaxis.set_ticks_position('bottom')
newax.xaxis.set_label_position('bottom')

newax.set_xticks(np.arange(-180,180,30))
newax.set_xlim([-180,180])
newax.set_ylim([-1,1])
newax.set_xlabel('phi_photon')
newax.set_ylabel('cos(theta)_photon')

fig.tight_layout()
plt.show()

In [ ]:
#NOTE: THE SCALE IS FAKE FOR PLOT 2 AND 6!

import plotly.graph_objects as go
from plotly.subplots import make_subplots

endrange=np.amax(MFPAD_RCR_cos)

fig = go.Figure()
fig = make_subplots(rows=6, cols=12, vertical_spacing=0.015)

for i in range(12):
    for j in range(6):
        fig.add_trace(go.Contour(x=phiM,y=cosM,z=MFPAD_RCR_cos[j+i*6].T, line_smoothing=0.7, legendgroup="no", colorscale="Viridis", showscale=False, showlegend=False, contours=dict(start=0., end=endrange, size=endrange/10)), row=j+1, col=i+1)
        # fig.add_trace(go.Contour(x=phiM,y=cosM,z=MFPAD_RCR_cos[j+i*6].T), row=j+1, col=i+1)
    
fig.update_layout(
    title=dict(
        text="MFPAD cos sorted" ,
        # y=0.9,
        x=0.5,
        xanchor="center",
        yanchor="top"),
    # coloraxis=dict(colorscale=Seismic_r),
    # showlegend=False,
    autosize=True,
    width=1000,
    # height=1500,
    margin=dict(l=0,r=0,b=25,t=25)
    )

fig.show()

In [ ]:
#NOTE: THE SCALE IS FAKE FOR PLOT 2 AND 6!

import plotly.graph_objects as go
from plotly.subplots import make_subplots

endrange=[]

for MFPAD in MFPAD_RCR_cos:
    endrange.append(np.amax(fugi.smoothgauss(MFPAD,0.3,0.6)))
endrange=np.amax(endrange)


fig = go.Figure()
fig = make_subplots(rows=6, cols=12, vertical_spacing=0.015)

for i in range(12):
    for j in range(6):
        fig.add_trace(go.Contour(x=phiM,y=cosM,z=fugi.smoothgauss(MFPAD_RCR_cos[j+i*6],0.3,0.6).T, line_smoothing=0.7, legendgroup="no", colorscale="Viridis", showscale=False, showlegend=False, contours=dict(start=0., end=endrange, size=endrange/10)), row=j+1, col=i+1)
    
fig.update_layout(
    title=dict(
        text="MFPAD smoothed cos sorted" ,
        # y=0.9,
        x=0.5,
        xanchor="center",
        yanchor="top"),
    # coloraxis=dict(colorscale=Seismic_r),
    # showlegend=False,
    autosize=True,
    width=1000,
    # height=1500,
    margin=dict(l=0,r=0,b=25,t=25)
    )

fig.show()

In [ ]:
fig,ax = plt.subplots(2,4, figsize=(20, 10))
ax[0][0].contourf(phiM,cosM,MFPAD_sumall_RCR.T, cmap="viridis")
ax[0][0].set_title("MFPAD RCR enant")
ax[0][1].contourf(phiM,cosM,MFPAD_sumall_SCR.T, cmap="viridis")
ax[0][1].set_title("MFPAD SCR enant")
ax[0][2].contourf(phiM,cosM,MFPAD_sumall_RCL.T, cmap="viridis")
ax[0][2].set_title("MFPAD RCL enant")
ax[0][3].contourf(phiM,cosM,MFPAD_sumall_SCL.T, cmap="viridis")
ax[0][3].set_title("MFPAD SCL enant")

ax[1][0].contourf(phiM,cosM,MFPAD_sumall_RCR_norm.T, cmap="viridis")
ax[1][0].set_title("norm")
ax[1][1].contourf(phiM,cosM,MFPAD_sumall_SCR_norm.T, cmap="viridis")
ax[1][1].set_title("norm")
ax[1][2].contourf(phiM,cosM,MFPAD_sumall_RCL_norm.T, cmap="viridis")
ax[1][2].set_title("norm")
ax[1][3].contourf(phiM,cosM,MFPAD_sumall_SCL_norm.T, cmap="viridis")
ax[1][3].set_title("norm")
plt.show()

In [ ]:
MFPAD_sumall_SR_single=MFPAD_sumall_SCL+np.flip(MFPAD_sumall_RCL,axis=0)+MFPAD_sumall_SCR+np.flip(MFPAD_sumall_RCR,axis=0)
MFPAD_sumall_SR_single_norm=MFPAD_sumall_SCL_norm+np.flip(MFPAD_sumall_RCL_norm,axis=0)+MFPAD_sumall_SCR_norm+np.flip(MFPAD_sumall_RCR_norm,axis=0)

fig,ax = plt.subplots(2,2, figsize=(11, 11))

ax[0][0].imshow(MFPAD_sumall_SR.T, extent=[phiM[0],phiM[-1],cosM[0],cosM[-1]], interpolation='quadric', cmap="viridis", aspect="auto")
ax[0][0].set_title("MFPAD SR enant")
ax[1][0].imshow(MFPAD_sumall_SR_norm.T, extent=[phiM[0],phiM[-1],cosM[0],cosM[-1]], interpolation='quadric', cmap="viridis", aspect="auto")
ax[1][0].set_title("norm")

ax[0][1].imshow(MFPAD_sumall_SR_single.T, extent=[phiM[0],phiM[-1],cosM[0],cosM[-1]], interpolation='quadric', cmap="viridis", aspect="auto")
ax[0][1].set_title("MFPAD sum")
ax[1][1].imshow(MFPAD_sumall_SR_single_norm.T, extent=[phiM[0],phiM[-1],cosM[0],cosM[-1]], interpolation='quadric', cmap="viridis", aspect="auto")
ax[1][1].set_title("norm")



plt.show()

In [ ]:
#cos(theta) seems inverted rispect to contour
fig = make_subplots(rows=1, cols=2,
                    subplot_titles=("MFPAD SR norm", "MFPAD sum norm"))

# fig.add_trace(go.Contour(z=param_matrix[0,:,0], x=go_phi, y=go_cos, line_smoothing=0, colorscale=Seismic_r, contours=dict(start=-0.2, end=0.2, size=0.04)), 1, 1)
fig.add_trace(go.Contour(x=phiM,y=cosM,z=MFPAD_sumall_SR_norm.T, line_smoothing=0.85, coloraxis="coloraxis1"), 1, 1)
fig.add_trace(go.Contour(x=phiM,y=cosM,z=MFPAD_sumall_SR_single_norm.T, line_smoothing=0.85, coloraxis="coloraxis2"), 1, 2)

fig.update_layout(
    coloraxis1=dict(colorscale='Viridis',
                    colorbar=dict(x=0.515, xanchor='right', thickness=10)),
    coloraxis2=dict(colorscale='Viridis',
                    colorbar=dict(x=1.0, xanchor='left', thickness=10)),
    # showlegend=False,
    autosize=False,
    width=1000,
    height=450,
    margin=dict(l=5,r=5,b=25,t=25)
    )

fig.show()